#**Méthodes basées sur la programmation dynamique**

Dans le cadre de l'apprentissage par renforcement, l'utilisation de méthodes de programmation dynamiques permet de calculer les solutions des fonctions de valeurs d'états et d'actions optimales. Cela nous permettra de déterminer par la suite la stratégie que l'agent doit suivre.

Ces méthodes fonctionnent lorsque:
- Le MDP possède un nombre fini et restreint d'états
- Le modèle de l'environnement est parfaitement connu

Nous allons étudier deux algorithmes qui sont:
- L'algorithme par **itération des stratégies** (Policy Iteration). Cet algorithme comprend deux étapes principales:
  - L'évaluation d'une stratégie (Policy Evaluation)
  - L'amélioration d'une stratégie (Policy Improvement)

- L'algorithme par **itération des valeurs** (Values Iteration).

##**5. Algorithme par itération des stratégies**

####**4.5. Exemple**

Reprenons l'exemple d'environnement proposé par OpenAI (Frozen Lake). Dans l'environnement Frozen Lake, l'agent peut se trouver sur 16 états et effectuer 4 actions par état : Haut, Bas, Gauche, Droite.

&nbsp;
<br/>
&nbsp;
<br/>
<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL10.png?raw=true" width="640"></center>
&nbsp;
<br/>
&nbsp;
<br/>

Dans cet environnement, il y a donc 4 trous (états 5, 7, 11 et 12) et un objectif (état 15). L'agent part de l'état 0 et son objectif est d'atteindre l'état 15 en trouvant le chemin le plus court. On assume que l'agent reçoit une récompense de +1 si il atteint l'objectif, de 0 s'il tombe dans un trou (et le jeu se termine) et dans les autres cas.

On considère qu'il y a 1/3 de chance que l'agent effectue correctement l'action choisie. Il y a donc 2/3 de chance que l'action soit défaillante. Mais attention, les autres actions possibles dans la liste des actions défaillantes sont celles qui sont les "plus proches" de l'action originale. Par exemple:

- Action choisie: "Droite":
  - 33% de chance d'aller à droite
  - 33% de chance d'aller en haut
  - 33% de chance d'aller en bas

- Action chosie "Haut":
  - 33% de chance d'aller en haut
  - 33% de chance d'aller à gauche
  - 33% de chance d'aller à droite   

...

Le programme suivant définit l'environnement Frozen Lake et affiche un apperçu de la table des transitions.

In [ ]:
from __future__ import print_function
import random
import gym
import numpy as np

# Chargement de l'environnement
env = gym.make('FrozenLake-v0')

# Remise à zéro de l'environnement et affichage de la carte
env.reset()                    

# Récupération de la table des transitions
P = env.env.P

# Affichage des valeurs contenues dans la table
for key, value in P.items():
    print (key, ": ", value)

**Algorithme d'évaluation d'une stratégie**

On définit maintenant l'algorithme permettant d'évaluer une stratégie (partie 2 de la figure ci-dessous) :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL30.png?raw=true" width="640"></center>

In [ ]:
# Fonction permettant d'évaluer une stratégie
# pi :  pi(a|s)
# P :   Table des transitions
# V_ :  Fonction des valeurs d'états issue de l'amélioration de la stratégie précédente
# gamma : Facteur de remise
# theta : Seuil déterminant la précision de l'évaluation

def evaluation_strategie(pi, P, V_, gamma=0.99, theta=1e-10):
    # Répéter tant que l'algorithme n'a pas convergé
    delta = 2*theta
    while delta > theta:
        # Initialise la fonction valeur d'état à 0
        V = np.zeros(len(P), dtype=np.float64)

        # Pour chaque état de l'environnement
        for s in range(len(P)):
            # Calcule la nouvelle valeur de l'état en cours à l'aide de la
            # fonction des valeurs d'états issus de l'amélioration de la
            # stratégie précédente
            for proba, etat_suivant, recompense, etat_terminal in P[s][pi(s)]:
                  V[s] += proba*(recompense + gamma*V_[etat_suivant])
            
        # Compare la fonction des valeurs des états obtenue avec la précédente
        delta = np.max(np.abs(V_ - V))
        print(delta)

        # Sauvegarde la fonction des valeurs des états pour la prochaine itération
        V_ = V.copy()
    return V

**Algorithme d'amélioration d'une stratégie**

On définit maintenant l'algorithme permettant l'amélioration d'une stratégie (partie 3 de la figure ci-dessous) :

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL30.png?raw=true" width="640"></center>

In [ ]:
# Fonction permettant d'améliorer une stratégie
#  pi :     Stratégie à améliorer
#  V :      Fonction de valeurs des états
#  P :      Table de transition
#  gamma :  Facteur de remise

def amelioration_strategie(pi, V, P, gamma=0.99):
    # Initalise la fonction des valeurs d'actions
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)

    # Initialise l'indicateur de strategie stabilisée
    strategie_stable = True

    # Pour chaque état de l'environnement
    for s in range(len(P)):
        # Pour chaque action disponnible sur cet état
        for a in range(len(P[s])):
            # Calcul de la valeur d'action Q(s,a)
            for proba, etat_suivant, recompense, etat_terminal in P[s][a]:
                Q[s][a] += proba*(recompense + gamma*V[etat_suivant])
            
        # Récupère l'action la plus forte sur cet état
        a_max = np.argmax(Q[s,:])

        # Compare cette action avec celle donnée par la stratégie à optimiser
        if a_max != pi(s):
          strategie_stable = False

    #compute new policy which is going to be compared. The iteration will brake if difference is under the threshold
    nouvelle_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return nouvelle_pi, strategie_stable

**Algorithme général d'itération des stratégies**

On définit maintenant l'algorithme général permettant de trouver une stratégie optimale par la méthode d'itération des stratégies:

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL30.png?raw=true" width="640"></center>

In [ ]:
# Algorithme d'itération des stratégies 
gamma = 0.99    # Facteur de remise
theta = 1e-10   # Seuil de précision de l'évaluation des stratégies

# Initialise une stratégie aléatoire pi(s)
actions_aleatoires = np.random.choice([0,1,2,3],len(P))
pi = lambda s: {s:a for s, a in enumerate(actions_aleatoires)}[s]               # pi(s) = action de la stratégie pi

strategie_stable = False

# Initialisation de la fonction des valeurs d'états à 0
V_ = np.zeros(len(P), dtype=np.float64)

while strategie_stable is False:
  print ("Stratégie : %s" %{s:pi(s) for s in range(len(P))})
  # Évaluation de la stratégie
  V = evaluation_strategie(pi, P, V_, gamma, theta)

  # Amélioration de la stratégie
  pi, strategie_stable = amelioration_strategie(pi, V, P, gamma)

**Affichage de la stratégie et de la fonction des valeurs des états**

In [ ]:
def affiche_strategie(pi, P, objectif, symboles_actions=('<', 'v', '>', '^'), n_cols=4, titre='Stratégie optimale:'):
    print(titre)
    arrs = {k:v for k,v in enumerate(symboles_actions)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if s == objectif:
            print("OBJ".rjust(6), end=" ")
        
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            if s != objectif:
                print("TROU".rjust(6), end=" ")
        else:
            #print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
            print(arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def affiche_fonction_valeurs_actions(V, P, objectif, n_cols=4, prec=3, titre="Fonction des valeurs d'états:"):
    print(titre)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        
        if s == objectif:
            print("OBJ".rjust(6), end=" ")
        
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            if s != objectif:
                print("TROU".rjust(6), end=" ")
        else:
            print('{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
affiche_strategie(pi,P,15)
print("")
affiche_fonction_valeurs_actions(V, P, 15)

####**4.6. Exemple avec des probabilités d'actions quasi-déterministes**

In [ ]:
from __future__ import print_function
import random
import gym
import numpy as np

# Chargement de l'environnement
env = gym.make('FrozenLake-v0')

# Remise à zéro de l'environnement et affichage de la carte
env.reset()                    

# Récupération de la table des transitions
P = env.env.P

In [ ]:
# Modification des récompenses
recompense_objectif, recompense_trou, recopense_autre = 1, -1, -0.04
objectif, trou = 15, [5, 7, 11, 12]
for s in range(len(P)):
    for a in range(len(P[s])):
        for t in range(len(P[s][a])):
            values = list(P[s][a][t])
            if values[1] == objectif:
                values[2] = recompense_objectif
                values[3] = False
            elif values[1] in trou:
                values[2] = recompense_trou
                values[3] = False
            else:
                values[2] = recopense_autre
                values[3] = False
            if s in trou or s == objectif:
                values[2] = 0
                values[3] = True
            P[s][a][t] = tuple(values)

# Modification de la table des transitions
prob_action, prob_glisse_1, prob_glisse_2 = 0.8, 0.1, 0.1
for s in range(len(P)):
    for a in range(len(P[s])):
        for t in range(len(P[s][a])):
            if P[s][a][t][0] == 1.0:
                continue
            values = list(P[s][a][t])
            if t == 0:
                values[0] = prob_glisse_1
            elif t == 1:
                values[0] = prob_action
            elif t == 2:
                values[0] = prob_glisse_2
            P[s][a][t] = tuple(values)

In [ ]:
# Affichage des valeurs contenues dans la table
for key, value in P.items():
    print (key, ": ", value)

In [ ]:
# Algorithme d'itération des stratégies 
gamma = 0.99    # Facteur de remise
theta = 1e-10   # Seuil de précision de l'évaluation des stratégies

# Initialise une stratégie aléatoire pi(s)
actions_aleatoires = np.random.choice([0,1,2,3],len(P))
pi = lambda s: {s:a for s, a in enumerate(actions_aleatoires)}[s]               # pi(s) = action de la stratégie pi

strategie_stable = False

# Initialisation de la fonction des valeurs d'états à 0
V_ = np.zeros(len(P), dtype=np.float64)

while strategie_stable is False:
  print ("Stratégie : %s" %{s:pi(s) for s in range(len(P))})
  # Évaluation de la stratégie
  V = evaluation_strategie(pi, P, V_, gamma, theta)

  # Amélioration de la stratégie
  pi, strategie_stable = amelioration_strategie(pi, V, P, gamma)

In [ ]:
affiche_strategie(pi,P,15)
print("")
affiche_fonction_valeurs_actions(V, P, 15)